In [29]:
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from joblib import parallel_backend
import json

In [30]:
df = pd.read_csv('S&P_stock_differences.csv', index_col=[0])

In [31]:
raw_stock_df = pd.read_csv('S&P_stock_prices.csv', index_col=[0])

In [32]:
with parallel_backend('threading', n_jobs=-1):
    for y in df.columns:
        val_dict = {}
        for x in df.columns:
            if x != y:
                subset = df[[y, x]]
                subset = subset.dropna(how='any')
                
                if (len(subset) <= 2):
                    val_dict[str(subset.iloc[:,1].name)] = {'cor_processed': 0}
                
                else:
                    try:
                            cor_val = spearmanr(np.array(subset.iloc[:, 0]), np.array(subset.iloc[:,1]), nan_policy='omit')
                            val_dict[str(x)] = {'cor_coef': round(cor_val[0], 6), 'p-val':round(cor_val[1], 6), 'cor_processed': 1}
                    except:
                            val_dict[str(subset.iloc[:,1].name)] = {'cor_processed': 0}
            else:
                pass
        
        name_dict = {str(y) : {'comparisons': val_dict, 'price': raw_stock_df[[y]].dropna(how='any').to_dict()[y]}}
        with open('./correlation-data/' + str(y) + '.json', 'w', encoding='utf-8') as f:
            json.dump(name_dict, f)
        del val_dict